In [1]:
import os
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from transformers import AutoTokenizer, AutoModel
from tqdm.auto import tqdm
import random
from collections import defaultdict, Counter
import warnings
warnings.filterwarnings('ignore')

CONFIGURATION


In [2]:
class Config:
    # Data
    TOTAL_SAMPLES = 50000
    TEST_SIZE = 0.05

    # Model
    HIDDEN_DIMS = [1024, 768, 512, 256]
    NUM_CLASSES = 10

    # Training
    BATCH_SIZE = 256
    EPOCHS = 30
    LEARNING_RATE = 1e-4
    WEIGHT_DECAY = 1e-5

    # Score distribution for balanced training
    SCORE_DISTRIBUTION = {
        1: 2.94, 2: 4.01, 3: 6.71, 4: 10.53, 5: 13.94,
        6: 10.78, 7: 15.97, 8: 22.87, 9: 11.19, 10: 1.07
    }

    # Paths
    TRAIN_DATA_PATH = 'train_data.json'
    TEST_DATA_PATH = 'test_data.json'
    METRIC_NAMES_PATH = 'metric_names.json'
    CHECKPOINT_DIR = 'checkpoints'
    SUBMISSION_PATH = 'submission.csv'

    # Random seed
    SEED = 42

    # Device
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def set_seed(seed):
    """Set random seed for reproducibility"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


DATA LOADING

In [3]:
def load_data():
    """Load training and test data"""
    print("="*70)
    print("STEP 1: Loading Data")
    print("="*70)

    with open(Config.TRAIN_DATA_PATH, 'r') as f:
        train_data = json.load(f)

    with open(Config.TEST_DATA_PATH, 'r') as f:
        test_data = json.load(f)

    with open(Config.METRIC_NAMES_PATH, 'r') as f:
        metric_names = json.load(f)

    print(f"Training samples: {len(train_data)}")
    print(f"Test samples: {len(test_data)}")
    print(f"Metrics: {len(metric_names)}")

    return train_data, test_data, metric_names

GEMMA EMBEDDINGS

In [ ]:
def check_hf_token():
    """Check if HuggingFace token is set"""
    
    token = "hf_your_actual_token_here"  # <-- Paste your token here
    return token


In [8]:
def load_gemma_model(device):
    """Load Gemma embedding model"""
    print("\nLoading Gemma model...")

    from huggingface_hub import login
    token = check_hf_token()
    login(token=token)

    tokenizer = AutoTokenizer.from_pretrained("google/embeddinggemma-300m")
    model = AutoModel.from_pretrained(
        "google/embeddinggemma-300m",
        torch_dtype=torch.float16 if device.type == 'cuda' else torch.float32
    )
    model = model.to(device).eval()

    return tokenizer, model

def encode_texts(texts, tokenizer, model, device, batch_size=16, desc="Encoding"):
    """Encode texts using Gemma model"""
    all_embeddings = []

    for i in tqdm(range(0, len(texts), batch_size), desc=desc):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding=True,
                          truncation=True, max_length=512).to(device)

        with torch.no_grad():
            outputs = model(**inputs)
            embeddings = outputs.last_hidden_state.mean(dim=1)

        all_embeddings.append(embeddings.cpu().numpy())

    return np.vstack(all_embeddings)

def format_text(sample):
    """Format sample for encoding"""
    sys_prompt = sample.get('system_prompt', '')
    if sys_prompt:
        return f"System: {sys_prompt}\nPrompt: {sample['user_prompt']}\nResponse: {sample['response']}"
    return f"Prompt: {sample['user_prompt']}\nResponse: {sample['response']}"

def generate_embeddings(train_data, test_data, metric_names, device):
    """Generate Gemma embeddings for all data"""
    print("\n" + "="*70)
    print("STEP 2: Generating Gemma Embeddings")
    print("="*70)

    tokenizer, model = load_gemma_model(device)

    # Encode metrics
    print("\nEncoding metric names...")
    metric_embeddings = encode_texts(metric_names, tokenizer, model, device,
                                     batch_size=32, desc="Metrics")

    # Encode training texts
    print("\nEncoding training data...")
    train_texts = [format_text(s) for s in train_data]
    train_text_embeddings = encode_texts(train_texts, tokenizer, model, device,
                                         batch_size=16, desc="Training")

    # Encode test texts
    print("\nEncoding test data...")
    test_texts = [format_text(s) for s in test_data]
    test_text_embeddings = encode_texts(test_texts, tokenizer, model, device,
                                        batch_size=16, desc="Test")

    # Clean up
    del model
    torch.cuda.empty_cache()

    # Clean and normalize embeddings
    print("\nNormalizing embeddings...")
    metric_embeddings = np.nan_to_num(metric_embeddings, 0)
    train_text_embeddings = np.nan_to_num(train_text_embeddings, 0)
    test_text_embeddings = np.nan_to_num(test_text_embeddings, 0)

    metric_scaler = StandardScaler()
    metric_embeddings = metric_scaler.fit_transform(metric_embeddings)

    text_scaler = StandardScaler()
    train_text_embeddings = text_scaler.fit_transform(train_text_embeddings)
    test_text_embeddings = text_scaler.transform(test_text_embeddings)

    print(f"\nEmbedding shapes:")
    print(f"  Metrics: {metric_embeddings.shape}")
    print(f"  Train text: {train_text_embeddings.shape}")
    print(f"  Test text: {test_text_embeddings.shape}")

    return metric_embeddings, train_text_embeddings, test_text_embeddings



In [10]:
# =============================================================================
# 3. DATA AUGMENTATION
# =============================================================================

def create_balanced_dataset(train_data, total_samples, score_distribution):
    """Create balanced augmented dataset"""
    print("\n" + "="*70)
    print("STEP 3: Creating Balanced Augmented Dataset")
    print("="*70)

    # Calculate target counts
    target_counts = {
        score: int(total_samples * pct / 100)
        for score, pct in score_distribution.items()
    }

    print(f"\nTarget: {total_samples} samples")
    print("Score distribution:")
    for score, count in target_counts.items():
        print(f"  Score {score}: {count:5d} ({count/total_samples*100:5.2f}%)")

    # Group by score
    by_score = defaultdict(list)
    for sample in train_data:
        score = int(float(sample['score']))
        if 1 <= score <= 10:
            by_score[score].append(sample)

    print("\nOriginal counts:")
    for score in range(1, 11):
        print(f"  Score {score}: {len(by_score[score])}")

    augmented = []

    # Create samples for each score
    for target_score in range(1, 11):
        target_count = target_counts[target_score]
        available = by_score.get(target_score, [])

        if len(available) == 0:
            # Borrow from nearby scores
            nearby = []
            for offset in [1, -1, 2, -2, 3, -3]:
                nearby_score = target_score + offset
                if 1 <= nearby_score <= 10:
                    nearby.extend(by_score.get(nearby_score, []))

            if len(nearby) > 0:
                sampled = random.choices(nearby, k=target_count)
                for s in sampled:
                    augmented.append({
                        'metric_name': s['metric_name'],
                        'user_prompt': s['user_prompt'],
                        'response': s['response'],
                        'system_prompt': s.get('system_prompt', ''),
                        'score': float(target_score)
                    })

        elif len(available) < target_count:
            # Oversample
            sampled = random.choices(available, k=target_count)
            for s in sampled:
                augmented.append({
                    'metric_name': s['metric_name'],
                    'user_prompt': s['user_prompt'],
                    'response': s['response'],
                    'system_prompt': s.get('system_prompt', ''),
                    'score': float(s['score'])
                })

        else:
            # Undersample
            sampled = random.sample(available, target_count)
            for s in sampled:
                augmented.append({
                    'metric_name': s['metric_name'],
                    'user_prompt': s['user_prompt'],
                    'response': s['response'],
                    'system_prompt': s.get('system_prompt', ''),
                    'score': float(s['score'])
                })

    random.shuffle(augmented)

    # Verify
    print("\nFinal distribution:")
    final_dist = Counter([int(s['score']) for s in augmented])
    for score in range(1, 11):
        count = final_dist[score]
        pct = count / len(augmented) * 100
        print(f"  Score {score}: {count:5d} ({pct:5.2f}%)")

    return augmented

def prepare_features(augmented_data, metric_embeddings, train_text_embeddings,
                    test_embeddings, metric_names, test_data, original_train_data):
    """
    Prepare feature matrices - CORRECTED VERSION
    Maps augmented samples back to their original embeddings
    """
    print("\n" + "="*70)
    print("Preparing Feature Matrices")
    print("="*70)

    metric_to_idx = {name: idx for idx, name in enumerate(metric_names)}

    # Create mapping: (metric, prompt, response) -> original embedding index
    print("\nCreating embedding lookup map...")

    original_map = {}
    for idx, sample in enumerate(original_train_data):
        # Use tuple as key (hashable)
        key = (
            sample['metric_name'],
            sample['user_prompt'],
            sample['response']
        )
        original_map[key] = idx

    print(f"Mapped {len(original_map)} unique training samples")

    # Prepare training features
    print("\nMapping augmented samples to embeddings...")

    train_metric_features = []
    train_text_features = []
    train_labels = []

    not_found_count = 0

    for sample in tqdm(augmented_data, desc="Processing"):
        # Metric embedding
        metric_idx = metric_to_idx[sample['metric_name']]
        train_metric_features.append(metric_embeddings[metric_idx])

        # Text embedding - find in original data
        key = (
            sample['metric_name'],
            sample['user_prompt'],
            sample['response']
        )

        if key in original_map:
            text_idx = original_map[key]
            train_text_features.append(train_text_embeddings[text_idx])
        else:
            # Fallback: use first embedding (shouldn't happen with correct augmentation)
            train_text_features.append(train_text_embeddings[0])
            not_found_count += 1

        train_labels.append(int(sample['score']))

    if not_found_count > 0:
        print(f"\nWarning: {not_found_count} samples not found in original data (using fallback)")

    train_metric_features = np.array(train_metric_features)
    train_text_features = np.array(train_text_features)
    train_labels = np.array(train_labels)

    # Test features
    test_metric_features = np.array([
        metric_embeddings[metric_to_idx[s['metric_name']]]
        for s in test_data
    ])
    test_text_features = test_embeddings

    print(f"\nFeature shapes:")
    print(f"  Train metric: {train_metric_features.shape}")
    print(f"  Train text: {train_text_features.shape}")
    print(f"  Train labels: {train_labels.shape}")
    print(f"  Test metric: {test_metric_features.shape}")
    print(f"  Test text: {test_text_features.shape}")

    return train_metric_features, train_text_features, train_labels, test_metric_features, test_text_features

# =============================================================================
# 4. MODEL DEFINITION
# =============================================================================

class MetricScoringModel(nn.Module):
    """Large capacity neural network for metric-based scoring"""

    def __init__(self, metric_dim, text_dim, hidden_dims, num_classes=10):
        super().__init__()

        input_dim = metric_dim + text_dim
        layers = []
        prev_dim = input_dim

        for i, hidden_dim in enumerate(hidden_dims):
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),
                nn.Dropout(0.3 if hidden_dim > 512 else 0.2)
            ])
            prev_dim = hidden_dim

        layers.append(nn.Linear(prev_dim, num_classes))

        self.network = nn.Sequential(*layers)
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def forward(self, metric_features, text_features):
        x = torch.cat([metric_features, text_features], dim=1)
        return self.network(x)

    def predict_scores(self, metric_features, text_features):
        logits = self.forward(metric_features, text_features)
        probs = torch.softmax(logits, dim=1)
        class_values = torch.arange(1, 11, device=probs.device, dtype=probs.dtype)
        return (probs * class_values).sum(dim=1)

class MetricDataset(Dataset):
    def __init__(self, metric_features, text_features, labels):
        self.metric_features = torch.FloatTensor(metric_features)
        self.text_features = torch.FloatTensor(text_features)
        self.labels = torch.LongTensor(labels - 1)  # 1-10 to 0-9

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.metric_features[idx], self.text_features[idx], self.labels[idx]

# =============================================================================
# 5. TRAINING
# =============================================================================

def train_epoch(model, loader, criterion, optimizer, device):
    """Train for one epoch"""
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    pbar = tqdm(loader, desc="Training")
    for metric_feat, text_feat, labels in pbar:
        metric_feat = metric_feat.to(device)
        text_feat = text_feat.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        logits = model(metric_feat, text_feat)
        loss = criterion(logits, labels)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        total_loss += loss.item()

        _, predicted = logits.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        pbar.set_postfix({'loss': f'{loss.item():.4f}',
                         'acc': f'{100.*correct/total:.2f}%'})

    return total_loss / len(loader), 100. * correct / total

def validate(model, loader, device):
    """Validate model"""
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for metric_feat, text_feat, labels in loader:
            metric_feat = metric_feat.to(device)
            text_feat = text_feat.to(device)

            scores = model.predict_scores(metric_feat, text_feat)

            all_preds.extend(scores.cpu().numpy())
            all_labels.extend((labels + 1).cpu().numpy())

    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    rmse = np.sqrt(np.mean((all_preds - all_labels)**2))

    return rmse, all_preds

def train_model(train_metric_feat, train_text_feat, train_labels, device):
    """Train the model"""
    print("\n" + "="*70)
    print("STEP 4: Training Model")
    print("="*70)

    # Split data
    X_m_tr, X_m_val, X_t_tr, X_t_val, y_tr, y_val = train_test_split(
        train_metric_feat, train_text_feat, train_labels,
        test_size=Config.TEST_SIZE, random_state=Config.SEED, stratify=train_labels
    )

    print(f"\nTrain: {len(y_tr)}, Validation: {len(y_val)}")

    # Create datasets
    train_dataset = MetricDataset(X_m_tr, X_t_tr, y_tr)
    val_dataset = MetricDataset(X_m_val, X_t_val, y_val)

    train_loader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE,
                             shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=Config.BATCH_SIZE*2,
                           num_workers=2)

    # Create model
    model = MetricScoringModel(
        metric_dim=train_metric_feat.shape[1],
        text_dim=train_text_feat.shape[1],
        hidden_dims=Config.HIDDEN_DIMS,
        num_classes=Config.NUM_CLASSES
    ).to(device)

    print(f"\nModel parameters: {sum(p.numel() for p in model.parameters()):,}")
    print(f"Architecture: Input → {' → '.join(map(str, Config.HIDDEN_DIMS))} → {Config.NUM_CLASSES}")

    # Training setup
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=Config.LEARNING_RATE,
                                 weight_decay=Config.WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=Config.EPOCHS)

    # Training loop
    os.makedirs(Config.CHECKPOINT_DIR, exist_ok=True)
    best_rmse = float('inf')

    print("\nTraining...")
    for epoch in range(Config.EPOCHS):
        print(f"\nEpoch {epoch+1}/{Config.EPOCHS}")

        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        val_rmse, val_preds = validate(model, val_loader, device)
        scheduler.step()

        print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.2f}%")
        print(f"Val - RMSE: {val_rmse:.4f}, Range: [{val_preds.min():.1f}, {val_preds.max():.1f}], "
              f"Mean: {val_preds.mean():.1f}, Std: {val_preds.std():.2f}")

        if val_rmse < best_rmse:
            best_rmse = val_rmse
            torch.save(model.state_dict(), f'{Config.CHECKPOINT_DIR}/best_model.pt')
            print(f"✓ Saved (Best RMSE: {best_rmse:.4f})")

    print(f"\nBest validation RMSE: {best_rmse:.4f}")

    # Load best model
    model.load_state_dict(torch.load(f'{Config.CHECKPOINT_DIR}/best_model.pt'))

    return model

# =============================================================================
# 6. PREDICTION & SUBMISSION
# =============================================================================

def generate_predictions(model, test_metric_feat, test_text_feat, device):
    """Generate predictions on test set"""
    print("\n" + "="*70)
    print("STEP 5: Generating Predictions")
    print("="*70)

    model.eval()

    test_m_tensor = torch.FloatTensor(test_metric_feat).to(device)
    test_t_tensor = torch.FloatTensor(test_text_feat).to(device)

    predictions = []

    print("\nPredicting...")
    with torch.no_grad():
        for i in tqdm(range(0, len(test_metric_feat), 512)):
            batch_m = test_m_tensor[i:i+512]
            batch_t = test_t_tensor[i:i+512]

            scores = model.predict_scores(batch_m, batch_t)
            predictions.extend(scores.cpu().numpy())

    predictions = np.array(predictions)
    predictions = np.clip(np.round(predictions), 1, 10)

    return predictions

def create_submission(predictions, test_data):
    """Create submission CSV file"""
    submission = pd.DataFrame({
        'ID': range(1, len(test_data) + 1),
        'score': predictions.astype(int).astype(float)
    })

    submission.to_csv(Config.SUBMISSION_PATH, index=False)

    print(f"\n✓ Submission saved to {Config.SUBMISSION_PATH}")



# =============================================================================
# MAIN PIPELINE
# =============================================================================

def main():
    """Main execution pipeline"""
    print("\n" + "="*70)
    print("METRIC-BASED TEXT SCORING - COMPLETE PIPELINE")
    print("="*70)
    print(f"Device: {Config.DEVICE}")


    # Set seed
    set_seed(Config.SEED)


    # Step 1: Load data
    train_data, test_data, metric_names = load_data()

    # Step 2: Generate embeddings
    metric_embs, train_text_embs, test_text_embs = generate_embeddings(
        train_data, test_data, metric_names, Config.DEVICE
    )

    # Step 3: Create augmented dataset
    augmented_data = create_balanced_dataset(
        train_data, Config.TOTAL_SAMPLES, Config.SCORE_DISTRIBUTION
    )

    # Prepare features (CORRECTED - now includes original_train_data)
    train_m_feat, train_t_feat, train_labels, test_m_feat, test_t_feat = prepare_features(augmented_data, metric_embs, train_text_embs,
                        test_text_embs, metric_names, test_data, train_data)

    # Step 4: Train model
    model = train_model(train_m_feat, train_t_feat, train_labels, Config.DEVICE)

    # Step 5: Generate predictions
    predictions = generate_predictions(model, test_m_feat, test_t_feat, Config.DEVICE)

    # Step 6: Create submission
    create_submission(predictions, test_data)

    print("\n" + "="*70)
    print("PIPELINE COMPLETED SUCCESSFULLY!")
    print("="*70)

if __name__ == "__main__":
    main()



METRIC-BASED TEXT SCORING - COMPLETE PIPELINE
Device: cuda
STEP 1: Loading Data
Training samples: 5000
Test samples: 3638
Metrics: 145

STEP 2: Generating Gemma Embeddings

Loading Gemma model...

Encoding metric names...


Metrics:   0%|          | 0/5 [00:00<?, ?it/s]


Encoding training data...


Training:   0%|          | 0/313 [00:00<?, ?it/s]


Encoding test data...


...

Training:   0%|          | 0/186 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7819d2cac7c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1664, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1647, in _shutdown_workers
Exception ignored in:     if w.is_alive():
<function _MultiProcessingDataLoaderIter.__del__ at 0x7819d2cac7c0> 
  Traceback (most recent call last):
   File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1664, in __del__
      self._shutdown_workers() 
^  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1647, in _shutdown_workers
^    ^if w.is_alive():^
^ ^ ^ ^ ^ ^ ^ ^^
^  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
^^    ^assert self._parent_pid == os.getpid(), 'can only test a child process'^^
^ ^ ^    ^^ 
   File "/usr/li

Train - Loss: 2.1890, Acc: 21.40%
Val - RMSE: 2.1571, Range: [6.1, 9.0], Mean: 6.2, Std: 0.32

Best validation RMSE: 2.1568

STEP 5: Generating Predictions

Predicting...


  0%|          | 0/8 [00:00<?, ?it/s]


✓ Submission saved to submission.csv

PIPELINE COMPLETED SUCCESSFULLY!
